In [ ]:
# Run inference

In [3]:
%%capture 
%run config.ipynb
%run ViT.ipynb

In [4]:
import torch 
from PIL import Image

In [ ]:
model = ViT(
    image_size=IMG_SIZE,
    patch_size=PATCH_SIZE,
    in_channels=IN_CHANNELS,
    n_head=N_HEAD, 
    d_model=D_MODEL, 
    ffn_hidden=FFN_HIDDEN, 
    mlp_hidden=MLP_HIDDEN, 
    n_layers=N_LAYERS, 
    class_num=CLASS_NUM, 
    device=device, 
    drop_prob=DROP_PROB,
)

# load model
model_path = model_dir / "model_20241128_094333_1"
model.load_state_dict(torch.load(model_path, weights_only=True, map_location=device))

model.eval()


In [ ]:
# prepare single image tensor for input 
img_inference_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])

img_path = data_dir / "image_name"

img = Image.open(image_path).convert("RGB")

img = img_inference_transform(img)

# set batch_size as 1 using unsqueeze
img.unsqueeze(0)
print(f'shape of img: {img.shape}')

In [5]:
# read the food101 classes.txt
class_file_path = rawdata_dir / "food-101" / "meta" / "classes.txt"
class_file = open(class_file_path, 'r')
class_list = [line for line in class_file]
class_dict = dict()
for idx, foodname in enumerate(class_list):
    class_dict[idx] = foodname

print(f'{class_dict}')

# define a helper function 
def tensor2foodid(tensor): 
    # convert tensor to foodid 
    # by picking index that has max value 
    # use argmax 
    food_id = torch.argmax(tensor, dim=-1)
    return food_id

def foodid2foodname(foodid):
    if foodid < 0 or foodid > max(class_dict.keys()):
        raise Exception("Invalid food Id for foodid2foodname function")
    return class_dict[foodid]
    

{0: 'apple_pie\n', 1: 'baby_back_ribs\n', 2: 'baklava\n', 3: 'beef_carpaccio\n', 4: 'beef_tartare\n', 5: 'beet_salad\n', 6: 'beignets\n', 7: 'bibimbap\n', 8: 'bread_pudding\n', 9: 'breakfast_burrito\n', 10: 'bruschetta\n', 11: 'caesar_salad\n', 12: 'cannoli\n', 13: 'caprese_salad\n', 14: 'carrot_cake\n', 15: 'ceviche\n', 16: 'cheesecake\n', 17: 'cheese_plate\n', 18: 'chicken_curry\n', 19: 'chicken_quesadilla\n', 20: 'chicken_wings\n', 21: 'chocolate_cake\n', 22: 'chocolate_mousse\n', 23: 'churros\n', 24: 'clam_chowder\n', 25: 'club_sandwich\n', 26: 'crab_cakes\n', 27: 'creme_brulee\n', 28: 'croque_madame\n', 29: 'cup_cakes\n', 30: 'deviled_eggs\n', 31: 'donuts\n', 32: 'dumplings\n', 33: 'edamame\n', 34: 'eggs_benedict\n', 35: 'escargots\n', 36: 'falafel\n', 37: 'filet_mignon\n', 38: 'fish_and_chips\n', 39: 'foie_gras\n', 40: 'french_fries\n', 41: 'french_onion_soup\n', 42: 'french_toast\n', 43: 'fried_calamari\n', 44: 'fried_rice\n', 45: 'frozen_yogurt\n', 46: 'garlic_bread\n', 47: 'gn

In [ ]:
def inference(model, img):
    with torch.no_grad():
        img = img.to(device)

        out, _ = model(img)
        foodid = tensor2foodid(out)
        foodname = foodid2foodname(foodid)

        print(f'{foodname}')
    return